In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('../main/레이블합친최종본.csv',encoding='cp949')
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%d-%b').dt.strftime('20%d%m')
df = df.drop('통계청 한국표준산업분류 10차(소분류)',axis=1)
df['거래소코드'] = df['거래소코드'].astype(str).str.zfill(6)
df = df[['회사명','거래소코드','회계년도','유동자산회전율','당좌자산구성비율','자기자본회전율','타인자본회전율',
         '재고자산_대_유동자산비율','매출액총이익율','총자본영업이익율','자기자본순이익율','설비투자효율',
         'CASH_FLOW_대_부채비율','배당금감소율여부','직원감소율여부','회생기업2']]
df = df.sort_values(by='회계년도')
df = df.reset_index(drop=True)
df

,회사명,거래소코드,회계년도,유동자산회전율,당좌자산구성비율,자기자본회전율,타인자본회전율,재고자산_대_유동자산비율,매출액총이익율,총자본영업이익율,자기자본순이익율,설비투자효율,CASH_FLOW_대_부채비율,배당금감소율여부,직원감소율여부,회생기업2
0,CMG제약,058820,201112,1.25,46.14,0.97,2.01,17.88,41.83,2.37,-8.28,35.858193,-46.08,0,0,0
1,이스타코,015020,201112,0.71,12.46,0.18,0.20,0.00,31.39,0.00,-6.59,0.000000,1.56,0,0,0
2,이수화학,005950,201112,3.48,39.89,4.57,3.05,22.03,7.27,6.60,4.28,72.865690,4.17,0,0,0
3,이수페타시스,007660,201112,1.85,46.30,2.39,2.52,27.93,10.33,5.39,5.18,135.343374,15.17,0,0,0
4,이수앱지스,086890,201112,0.29,48.40,0.28,0.79,16.83,54.21,0.00,-29.31,0.000000,-111.45,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8787,케일럼,258610,201912,0.76,47.08,0.71,0.92,1.85,16.98,1.10,-4.12,9.914524,7.75,0,0,0
8788,켐트로닉스,089010,201912,3.26,32.06,4.53,2.32,31.54,17.06,10.01,18.63,65.126119,14.30,0,0,0
8789,일진홀딩스,015860,201912,1.81,35.30,1.86,1.34,20.88,16.37,2.75,4.65,35.230020,6.75,0,0,0
8790,코데즈컴바인,047770,201912,1.98,29.02,0.92,1.23,16.63,60.69,6.64,17.73,0.000000,13.72,0,0,0


In [80]:
medi = lambda x: x.fillna(x.mean())
df = df.groupby('회사명').apply(medi)
df.isna().sum()

회사명                 0
거래소코드               0
회계년도                0
유동자산회전율             0
당좌자산구성비율            0
자기자본회전율             0
타인자본회전율             0
재고자산_대_유동자산비율       0
매출액총이익율             0
총자본영업이익율            0
자기자본순이익율            0
설비투자효율              0
CASH_FLOW_대_부채비율    0
배당금감소율여부            0
직원감소율여부             0
회생기업2               0
dtype: int64

### 피쳐와 레이블 나누기

In [81]:
data = df.iloc[:,1:15]
target = df.iloc[:,15]

In [82]:
data

,거래소코드,회계년도,유동자산회전율,당좌자산구성비율,자기자본회전율,타인자본회전율,재고자산_대_유동자산비율,매출액총이익율,총자본영업이익율,자기자본순이익율,설비투자효율,CASH_FLOW_대_부채비율,배당금감소율여부,직원감소율여부
0,058820,201112,1.25,46.14,0.97,2.01,17.88,41.83,2.37,-8.28,35.858193,-46.08,0,0
1,015020,201112,0.71,12.46,0.18,0.20,0.00,31.39,0.00,-6.59,0.000000,1.56,0,0
2,005950,201112,3.48,39.89,4.57,3.05,22.03,7.27,6.60,4.28,72.865690,4.17,0,0
3,007660,201112,1.85,46.30,2.39,2.52,27.93,10.33,5.39,5.18,135.343374,15.17,0,0
4,086890,201112,0.29,48.40,0.28,0.79,16.83,54.21,0.00,-29.31,0.000000,-111.45,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8787,258610,201912,0.76,47.08,0.71,0.92,1.85,16.98,1.10,-4.12,9.914524,7.75,0,0
8788,089010,201912,3.26,32.06,4.53,2.32,31.54,17.06,10.01,18.63,65.126119,14.30,0,0
8789,015860,201912,1.81,35.30,1.86,1.34,20.88,16.37,2.75,4.65,35.230020,6.75,0,0
8790,047770,201912,1.98,29.02,0.92,1.23,16.63,60.69,6.64,17.73,0.000000,13.72,0,0


In [84]:
target

0       0
1       0
2       0
3       0
4       0
       ..
8787    0
8788    0
8789    0
8790    0
8791    0
Name: 회생기업2, Length: 8792, dtype: int64

### 데이터 분리

In [85]:
# 기간을 얼마나 나누게 될까요~?
x_train = data.loc[:5641]
x_test = data.loc[5642:]
y_train = target.loc[:5641]
y_test = target.loc[5642:]

print('X 테스트 데이터 :')
print(x_test)
print('y 테스트 데이터 :')
print(list(y_test))

X 테스트 데이터 :
       거래소코드    회계년도  유동자산회전율  당좌자산구성비율  자기자본회전율  타인자본회전율  재고자산_대_유동자산비율  \
5642  187220  201712     1.63     24.91     0.68     1.14           0.00   
5643  090080  201712     5.10     27.35     5.11     3.00          23.00   
5644  086670  201712     1.18     30.72     1.13     1.62          44.64   
5645  267260  201712     0.53     54.08     0.63     0.55          23.99   
5646  200880  201712     2.01     51.07     2.97     1.81          10.11   
...      ...     ...      ...       ...      ...      ...            ...   
8787  258610  201912     0.76     47.08     0.71     0.92           1.85   
8788  089010  201912     3.26     32.06     4.53     2.32          31.54   
8789  015860  201912     1.81     35.30     1.86     1.34          20.88   
8790  047770  201912     1.98     29.02     0.92     1.23          16.63   
8791  046210  201912     0.68     40.70     0.44     1.70          12.33   

      매출액총이익율  총자본영업이익율  자기자본순이익율     설비투자효율  CASH_FLOW_대_부채비율  배당금감소율여부  \

In [86]:
# 피처, 레이블 데이터 분포 비율

train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(x_train.shape, x_test.shape))

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/train_cnt)

학습 세트 Shape:(5642, 14), 테스트 세트 Shape:(3150, 14)
 학습 세트 레이블 값 분포 비율
0    0.968983
1    0.031017
Name: 회생기업2, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.548742
1    0.009571
Name: 회생기업2, dtype: float64


In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


lg = LogisticRegression()
param_grid = [{'C':np.linspace(0.1,10,100)}]
gs = GridSearchCV(estimator=lg, param_grid=param_grid, scoring='accuracy',cv=5,n_jobs=1)
gs.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=1,
             param_grid=[{'C': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,
        3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,
        5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,  6.6,
        6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,  7.7,
        7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,  8.8,
        8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,  9.9,
       10. ])}],
             scoring='accuracy')

In [90]:
from sklearn.metrics import accuracy_score
best_est = gs.best_estimator_
predictions = best_est.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print('GridSearchCV 최적 하이퍼 파라미터 :', gs.best_params_)
print('GridSearchCV 최고 정확도 :{0:.4f}'.format(gs.best_score_))
print('테스트 세트에서의 LogisticRegression 정확도 : {0:.4f}'.format(accuracy))

GridSearchCV 최적 하이퍼 파라미터 : {'C': 1.2000000000000002}
GridSearchCV 최고 정확도 :0.9688
테스트 세트에서의 LogisticRegression 정확도 : 0.9825


### 교차검증 전 정확도 확인

In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, accuracy_score, roc_auc_score, recall_score, \
    precision_score, confusion_matrix
import time
import warnings
warnings.filterwarnings('ignore')

lr_reg = LogisticRegression()

parameters = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],\
            'C':[0.01, 0.1, 1, 5, 10],\
            'solver':['lbfgs', 'liblinear']}
def best_lrparam(clf, param, scoring_name, cv_num):
    b_params= []
    
    grid_cv = GridSearchCV(clf, param_grid=param, scoring=scoring_name, cv=cv_num,)
    grid_cv.fit(x_train, y_train)
    
    b_params.append(grid_cv.best_params_)
    
    print('GridSearchCV 최적 하이퍼 파라미터 :', grid_cv.best_params_)
    print('\nGridSearchCV 최고 정확도 :{0:.4f}'.format(grid_cv.best_score_))
    best_est = grid_cv.best_estimator_
    
    # GridSearchCV의 최적 하이퍼 마라미터로 학습된 Estimator로 예측 및 평가 수행
    predictions = best_est.predict(x_test)
    accuracy = accuracy_score(y_test, predictions)
    print('테스트 세트에서의 LogisticRegression 정확도 : {0:.4f}'.format(accuracy))
    
    return b_params

### 데이터검증

for i in range(1, STEPS):
    col_name = 'd{}'.format(i)
    df[col_name] = df['d0'].shift(periods=-1 * i)
df = df.dropna()

In [ ]:
X = df.iloc[:, :TRAIN_STEPS]
y = df.iloc[:, TRAIN_STEPS:]

X_train = X.iloc[:SPLIT_IDX, :]
y_train = y.iloc[:SPLIT_IDX, :]

X_test = X.iloc[SPLIT_IDX:, :]
y_test = y.iloc[SPLIT_IDX:, :]

In [ ]:
def build_model(_alpha, _l1_ratio):
    estimator = ElasticNet(
        alpha=_alpha,
        l1_ratio=_l1_ratio,
        fit_intercept=True,
        normalize=False,
        precompute=False,
        max_iter=16,
        copy_X=True,
        tol=0.1,
        warm_start=False,
        positive=False,
        random_state=None,
        selection='random'
    )

    return MultiOutputRegressor(estimator, n_jobs=4)

In [ ]:
model = build_model(_alpha=1.0, _l1_ratio=0.3)
kfcv = KFold(n_splits=5)
scores = cross_val_score(model, X_train, y_train, cv=kfcv, scoring=r2)
print("Loss: {0:.3f} (+/- {1:.3f})".format(scores.mean(), scores.std()))

model = build_model(_alpha=1.0, _l1_ratio=0.3)
tscv = TimeSeriesSplit(n_splits=5)
scores = cross_val_score(model, X_train, y_train, cv=tscv, scoring=r2)
print("Loss: {0:.3f} (+/- {1:.3f})".format(scores.mean(), scores.std()))

In [ ]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]